# Residual Diagnostics with PanelBox v0.7.0

**NEW in v0.7.0**: Complete residual diagnostics with ResidualResult container

This notebook demonstrates:
1. Creating ResidualResult from fitted models
2. Running 4 diagnostic tests (Shapiro-Wilk, Jarque-Bera, Durbin-Watson, Ljung-Box)
3. Analyzing summary statistics
4. Generating interactive HTML reports
5. Interpreting diagnostic results

---

## Setup

In [1]:
import panelbox as pb
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print(f"PanelBox version: {pb.__version__}")

PanelBox version: 0.8.0


## 1. Load Data and Fit Model

We'll use the Grunfeld investment dataset.

In [2]:
# Load Grunfeld dataset
data = pb.load_grunfeld()

print(f"Dataset shape: {data.shape}")
print(f"Panel structure: {data['firm'].nunique()} firms, {data['year'].nunique()} years")
data.head()

Dataset shape: (200, 5)
Panel structure: 10 firms, 20 years


,firm,year,invest,value,capital
0,1,1935,317.6,3078.5,2.8
1,1,1936,391.8,4661.7,52.6
2,1,1937,410.6,5387.1,156.9
3,1,1938,257.7,2792.2,209.2
4,1,1939,330.8,4313.2,203.4


In [3]:
# Create experiment
experiment = pb.PanelExperiment(
    data=data,
    formula="invest ~ value + capital",
    entity_col="firm",
    time_col="year"
)

# Fit Fixed Effects model
experiment.fit_model('fixed_effects', name='fe')

print("✓ Fixed Effects model fitted")

Fitting fixed_effects model 'fe'...
✅ Model 'fe' fitted successfully
✓ Fixed Effects model fitted


## 2. Analyze Residuals (NEW in v0.7.0!)

The `analyze_residuals()` method creates a ResidualResult container with:
- Residuals and fitted values
- Standardized residuals
- 4 diagnostic tests
- Summary statistics

In [4]:
# Analyze residuals - NEW in v0.7.0!
residual_result = experiment.analyze_residuals('fe')

print("✓ ResidualResult created")
print(f"  Number of observations: {len(residual_result.residuals)}")
print(f"  Number of tests available: 4")

✓ ResidualResult created
  Number of observations: 200
  Number of tests available: 4


## 3. Diagnostic Tests

ResidualResult includes 4 diagnostic tests:
1. **Shapiro-Wilk** - Test for normality
2. **Jarque-Bera** - Alternative normality test
3. **Durbin-Watson** - Test for autocorrelation
4. **Ljung-Box** - Test for serial correlation (10 lags)

### 3.1 Shapiro-Wilk Test (Normality)

In [5]:
stat, pvalue = residual_result.shapiro_test

print("Shapiro-Wilk Test for Normality")
print("=" * 40)
print(f"Statistic: {stat:.6f}")
print(f"P-value: {pvalue:.6f}")
print(f"\nInterpretation:")
if pvalue > 0.05:
    print("  ✓ Residuals appear to be normally distributed (p > 0.05)")
else:
    print("  ✗ Residuals deviate from normality (p < 0.05)")
print(f"\nNote: Statistic close to 1.0 indicates normality")

Shapiro-Wilk Test for Normality
Statistic: 0.888696
P-value: 0.000000

Interpretation:
  ✗ Residuals deviate from normality (p < 0.05)

Note: Statistic close to 1.0 indicates normality


### 3.2 Jarque-Bera Test (Normality)

In [6]:
stat, pvalue = residual_result.jarque_bera

print("Jarque-Bera Test for Normality")
print("=" * 40)
print(f"Statistic: {stat:.6f}")
print(f"P-value: {pvalue:.6f}")
print(f"\nInterpretation:")
if pvalue > 0.05:
    print("  ✓ Residuals appear to be normally distributed (p > 0.05)")
else:
    print("  ✗ Residuals deviate from normality (p < 0.05)")
print(f"\nNote: Tests for skewness and kurtosis")

Jarque-Bera Test for Normality
Statistic: 155.841347
P-value: 0.000000

Interpretation:
  ✗ Residuals deviate from normality (p < 0.05)

Note: Tests for skewness and kurtosis


### 3.3 Durbin-Watson Test (Autocorrelation)

In [7]:
dw = residual_result.durbin_watson

print("Durbin-Watson Test for Autocorrelation")
print("=" * 40)
print(f"Statistic: {dw:.6f}")
print(f"\nInterpretation:")
if dw < 1.5:
    print(f"  ✗ Positive autocorrelation detected (DW < 1.5)")
elif dw > 2.5:
    print(f"  ✗ Negative autocorrelation detected (DW > 2.5)")
else:
    print(f"  ✓ No significant autocorrelation (1.5 < DW < 2.5)")
print(f"\nNote: DW ≈ 2 indicates no autocorrelation")

Durbin-Watson Test for Autocorrelation
Statistic: 1.007948

Interpretation:
  ✗ Positive autocorrelation detected (DW < 1.5)

Note: DW ≈ 2 indicates no autocorrelation


### 3.4 Ljung-Box Test (Serial Correlation)

In [8]:
stat, pvalue = residual_result.ljung_box

print("Ljung-Box Test for Serial Correlation")
print("=" * 40)
print(f"Statistic (10 lags): {stat:.6f}")
print(f"P-value: {pvalue:.6f}")
print(f"\nInterpretation:")
if pvalue > 0.05:
    print("  ✓ No serial correlation detected (p > 0.05)")
else:
    print("  ✗ Serial correlation present (p < 0.05)")
print(f"\nNote: Tests autocorrelation up to 10 lags")

Ljung-Box Test for Serial Correlation
Statistic (10 lags): 65.345964
P-value: 0.000000

Interpretation:
  ✗ Serial correlation present (p < 0.05)

Note: Tests autocorrelation up to 10 lags


## 4. Summary Statistics

ResidualResult provides easy access to summary statistics:

In [9]:
print("Residual Summary Statistics")
print("=" * 40)
print(f"Mean: {residual_result.mean:.6f} (should be ≈ 0)")
print(f"Std Dev: {residual_result.std:.6f}")
print(f"Skewness: {residual_result.skewness:.6f} (should be ≈ 0 for normality)")
print(f"Kurtosis: {residual_result.kurtosis:.6f} (should be ≈ 3 for normality)")
print(f"Min: {residual_result.min:.6f}")
print(f"Max: {residual_result.max:.6f}")
print(f"\nInterpretation:")
if abs(residual_result.mean) < 0.01:
    print("  ✓ Mean close to zero")
if abs(residual_result.skewness) < 0.5:
    print("  ✓ Low skewness")
if 2 < residual_result.kurtosis < 4:
    print("  ✓ Kurtosis close to normal (3)")

Residual Summary Statistics
Mean: -0.000000 (should be ≈ 0)
Std Dev: 48.500801
Skewness: 0.022064 (should be ≈ 0 for normality)
Kurtosis: 4.324236 (should be ≈ 3 for normality)
Min: -189.594628
Max: 200.992913

Interpretation:
  ✓ Mean close to zero
  ✓ Low skewness


## 5. Complete Summary

The `summary()` method provides a comprehensive text report:

In [10]:
print(residual_result.summary())

Residual Diagnostic Analysis

Summary Statistics:
--------------------------------------------------
Observations:               200
Mean:                   -0.0000
Std. Deviation:         48.5008
Min:                  -189.5946
Max:                   200.9929
Skewness:                0.0221
Kurtosis:                4.3242

Diagnostic Tests:
--------------------------------------------------
Shapiro-Wilk (Normality):        W = 0.889, p = 0.000 ✗ FAIL
Jarque-Bera (Normality):         JB = 155.84, p = 0.000 ✗ FAIL
Durbin-Watson (Autocorrelation): DW = 1.008 (Positive autocorrelation)
Ljung-Box (Autocorrelation):     Q = 65.35, p = 0.000 ✗ FAIL

Interpretation:
--------------------------------------------------
✗ Residuals may not be normally distributed
✗ Autocorrelation may be present
⚠ Some model assumptions may be violated


## 6. Generate HTML Report

Generate an interactive HTML report with diagnostic charts:

In [11]:
# Generate HTML report
html_path = residual_result.save_html(
    'residuals_diagnostics_report.html',
    test_type='residuals'
)

print(f"✓ HTML report saved to: {html_path}")
print(f"  Open in browser to see interactive visualizations")

✓ HTML report saved to: residuals_diagnostics_report.html
  Open in browser to see interactive visualizations


## 7. Export to JSON

Export results for programmatic analysis:

In [12]:
# Export to JSON
json_path = residual_result.save_json('residuals_diagnostics.json')

print(f"✓ JSON export saved to: {json_path}")

# Load and inspect
import json
with open(json_path) as f:
    data = json.load(f)

print(f"\nJSON structure:")
print(f"  - tests: {list(data['tests'].keys())}")
print(f"  - summary: {list(data['summary'].keys())}")
print(f"  - metadata: {list(data['_metadata'].keys())}")

✓ JSON export saved to: residuals_diagnostics.json

JSON structure:
  - tests: ['shapiro_wilk', 'jarque_bera', 'durbin_watson', 'ljung_box']
  - summary: ['n_obs', 'mean', 'std', 'min', 'max', 'skewness', 'kurtosis']
  - metadata: ['timestamp', 'class', 'experiment_formula', 'model_name']


## 8. Compare Multiple Models

Analyze residuals from different models:

In [13]:
# Fit multiple models
experiment.fit_model('pooled_ols', name='pooled')
experiment.fit_model('random_effects', name='re')

# Analyze residuals for each
models = ['pooled', 'fe', 're']
results = {}

for model_name in models:
    results[model_name] = experiment.analyze_residuals(model_name)

print("Residual Diagnostics Comparison")
print("=" * 60)
print(f"{'Model':<12} {'Mean':>10} {'Std':>10} {'DW':>10} {'SW p-val':>12}")
print("-" * 60)

for model_name, res in results.items():
    _, sw_p = res.shapiro_test
    print(f"{model_name:<12} {res.mean:>10.4f} {res.std:>10.4f} {res.durbin_watson:>10.4f} {sw_p:>12.4f}")

Fitting pooled_ols model 'pooled'...
✅ Model 'pooled' fitted successfully
Fitting random_effects model 're'...
✅ Model 're' fitted successfully
Residual Diagnostics Comparison
Model              Mean        Std         DW     SW p-val
------------------------------------------------------------
pooled           0.0000    94.4176     0.3221       0.0000
fe              -0.0000    48.5008     1.0079       0.0000
re               0.0000    96.5603     0.3042       0.0000


## 9. Standardized Residuals

Access standardized residuals for outlier detection:

In [14]:
# Get standardized residuals
std_resid = residual_result.standardized_residuals

# Find potential outliers (|z| > 2)
outliers = np.abs(std_resid) > 2
n_outliers = np.sum(outliers)

print(f"Outlier Detection (|standardized residual| > 2)")
print("=" * 40)
print(f"Number of outliers: {n_outliers} ({100*n_outliers/len(std_resid):.1f}%)")
print(f"Expected (normal): ~5%")

if n_outliers > 0:
    print(f"\nOutlier indices: {np.where(outliers)[0][:10]}...")

Outlier Detection (|standardized residual| > 2)
Number of outliers: 18 (9.0%)
Expected (normal): ~5%

Outlier indices: [ 2  4 11 14 16 18 19 23 24 29]...


## 10. Interpretation Guide

### Normality Tests (Shapiro-Wilk, Jarque-Bera)
- **p > 0.05**: Residuals appear normal ✓
- **p < 0.05**: Residuals deviate from normality ✗
- **Impact**: Non-normal residuals may affect hypothesis tests
- **Solution**: Consider robust standard errors

### Autocorrelation Tests (Durbin-Watson, Ljung-Box)
- **DW ≈ 2**: No autocorrelation ✓
- **DW < 1.5 or > 2.5**: Autocorrelation present ✗
- **Impact**: Standard errors may be biased
- **Solution**: Use HAC standard errors (Newey-West)

### Summary Statistics
- **Mean ≈ 0**: Good model fit ✓
- **Skewness ≈ 0**: Symmetric distribution ✓
- **Kurtosis ≈ 3**: Normal tails ✓

---

## Summary

**ResidualResult** (NEW in v0.7.0) provides:

✅ **4 Diagnostic Tests**:
- Shapiro-Wilk (normality)
- Jarque-Bera (normality)
- Durbin-Watson (autocorrelation)
- Ljung-Box (serial correlation)

✅ **Summary Statistics**:
- Mean, std, skewness, kurtosis, min, max
- Standardized residuals for outlier detection

✅ **Easy Workflows**:
- One-liner: `experiment.analyze_residuals('model_name')`
- HTML reports with interactive charts
- JSON export for further analysis

✅ **Professional Output**:
- Text summary with interpretation
- Interactive HTML report
- Integration with visualization system

---

**Next Steps**:
1. Open the HTML report in your browser
2. Inspect the JSON file for programmatic access
3. Compare residuals across different models
4. Use diagnostic results to refine your model